In [1]:
import pandas as pd
from streamlit_assets.components import get_recos_users
from sklearn.preprocessing import LabelEncoder
from surprise import SVD, NMF, KNNBasic, SVDpp
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
import numpy as np

### Exploring metacritic data for recos

In [2]:
meta_df = pd.read_csv('data/metacritic.csv').dropna(subset='critic_meta_score')
#meta_df = meta_df.rename(columns={"title":"item_id"})

meta_df.describe()


,rank,critic_meta_score,meta_score
count,52689.000000,52689.000000,52689.000000
mean,1554.035472,65.396136,65.277990
std,899.226203,21.106417,14.802795
min,1.000000,0.000000,13.000000
25%,790.000000,50.000000,56.000000
50%,1551.000000,70.000000,66.000000
75%,2318.000000,80.000000,76.000000
max,3139.000000,100.000000,99.000000


In [3]:
id_maker = LabelEncoder()
meta_df['user_id'] = id_maker.fit_transform(meta_df['author'])
meta_df['item_id'] = meta_df['title']
meta_df['rating'] = meta_df['critic_meta_score'].apply(int)

reviews = meta_df[['user_id','item_id','rating']]

In [14]:
tv_list = ['Breaking Bad: Season 5','Rectify: Season 4','Better Call Saul: Season 5','The Leftovers: Season 1','Lodge 49','The Underground Railroad']
#new_ids = meta_df.loc[meta_df['title'].isin(tv_list), 'item_id'].unique()

In [15]:
df_recos = get_recos_users(tv_list,reviews, filter_items=True,filter_n=20, default_rating=90)

C:\git\reco_system\streamlit_assets\components.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reviews = df.append(new_df).reset_index(drop=True)


In [16]:
df_merged = meta_df.merge(df_recos, on='item_id').drop_duplicates(subset='item_id').sort_values(by='user match', ascending=False).reset_index(drop=True)
df_merged[['title','summary','user match','meta_score']].head(20)

,title,summary,user match,meta_score
0,I May Destroy You: Season 1,"Arabella (Michaela Coel), an up-and-coming wri...",91.852374,86
1,Atlanta: Season 2,The second season subtitled the Atlanta Robbin...,90.905189,97
2,Atlanta: Season 3,"Earn, Alfred, Darius and Van are on tour in Eu...",90.707565,91
3,Transparent: Season 2,Maura's (Jeffrey Tambor) journey continues in ...,88.801758,94
4,The Beatles: Get Back: Season 1,Peter Jackson's three-part Beatles documentary...,88.443264,85
5,Orange is the New Black: Season 2,Season two finds Piper (Taylor Schilling) spen...,88.321096,89
6,The Americans: Season 3,The third season picks up late in 1982 with El...,87.489114,92
7,Succession: Season 3,Logan looks to firm up his position after Kend...,87.182786,92
8,Severance: Season 1,Mark Scout (Adam Scott) is one of several empl...,87.148765,83
9,Sharp Objects: Season 1,St. Louis crime reporter Camille Preaker (Amy ...,87.016063,78
